In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
import pandas as pd
from io import StringIO
import os

# Function to run queries
def run_query(query):
    # Prepare the input with context and query
    input_data = {
        "context": docs,
        "query": query
    }
    # Invoke the chain with the input data
    response = chain.invoke(input_data)
    return response

# Function to convert Txt to CSV
def syllabus_conversion_query(conversion_query):
    # Prepare the input with context and query
    conversion_input_data = {
        "context": syllabus_array,
        "query": conversion_query
    }
    # Invoke the chain with the input data
    response = syllabus_conversion_chain.invoke(conversion_input_data)
    return response

def weekly_schedule_conversion_query(conversion_query):
    # Prepare the input with context and query
    conversion_input_data = {
        "context": weekly_schedule_array,
        "query": conversion_query
    }
    # Invoke the chain with the input data
    response = weekly_schedule_conversion_chain.invoke(conversion_input_data)
    return response
# Function to clean and convert CSV strings to DataFrames
def clean_csv_string(csv_string):
    # Remove extra text and code block formatting
    start = csv_string.find("```csv")
    end = csv_string.rfind("```")
    if start != -1 and end != -1:
        csv_string = csv_string[start + len("```csv"):end].strip()
    return csv_string

# Function to convert cleaned CSV strings to DataFrames
def csv_string_to_dataframe(csv_string):
    dataframe = pd.read_csv(StringIO(csv_string))
    return dataframe


# Initialize the language model
openai.api_key = os.environ["OPENAI_API_KEY"]
llm = ChatOpenAI(temperature=0, model='gpt-4o', api_key=openai.api_key)

# Create the prompt templates
prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You have access to the class syllabus, weekly schedule, student grades, and the professor information. Answer the following question by searching through these data sources: {context} Make sure to fully read each data source (every row and column) and memorize the position of each value and their respective row column. Extra notes to remember: Economics is not considered a Mathematics course in this scenario, when asked what days and periods a subject is taught in, a subject can't be taught more than once per day (as in there can't be multiple sections on the same day), so go row by row, find the subject and note down its location (row and column) for your answer, on top of this search by subject not teacher name as there appears to be errors with grabbing values by teacher name, please double check each period value's correct day of the week, when asked a question regarding all the teachers of a subject, thoroughly make sure you go through the entire row of that subject in the Weekly Schedule and log each period's instructor down as an instructor of the subject."), 
    ("user", "{query}")
    ]
)
syllabus_conversion_template = ChatPromptTemplate.from_messages(
    [("system", "Please convert the following Txt file {context} to be formatted as a CSV file, output a comma separated string (CSV), sort the CSV only by columns with the value below being the syllabus from the text, (the columns are in this order: Chemistry, Mathematics, Physical Education, Foreign Language, Art, Economics, Physics, English Language)"), 
    ("user", "{query}")
    ]
)
weekly_schedule_conversion_template = ChatPromptTemplate.from_messages(
    [("system", "Please convert the following Txt file {context} to be formatted as a CSV file, output a comma separated string (CSV), sort the CSV based on each row representing a specific subject (this is the rows: Chemistry, Mathematics, Physical Education, Foreign Language, Art, Economics, Physics, English Language), and the columns correspond to different days of the week (Monday to Friday), each value is in this format (# Period) Subject (Full Professor Name) for example: (3rd Period) Chemistry (Jasmine Gomez) after logging all the info of the Chemistry Period on Monday."), 
    ("user", "{query}")
    ]
)

# Load text files as documents
syllabus_content = open('Syllabus.txt').read()
weekly_schedule_content = open('Weekly Schedule.txt').read()

# Create Document objects
syllabus_doc = Document(page_content=syllabus_content, name='Class Syllabus')
weekly_schedule_doc = Document(page_content=weekly_schedule_content, name='Weekly Schedule')

syllabus_array = [syllabus_doc]
weekly_schedule_array = [weekly_schedule_doc]

# Create the document and conversion chain
chain = create_stuff_documents_chain(llm, prompt_template)
syllabus_conversion_chain = create_stuff_documents_chain(llm, syllabus_conversion_template)
weekly_schedule_conversion_chain = create_stuff_documents_chain(llm, weekly_schedule_conversion_template)

# Convert text files to DataFrames
syllabus_csv_string = syllabus_conversion_query("Please follow the system instructions to convert the Txt file to a CSV.")
weekly_schedule_csv_string = weekly_schedule_conversion_query("Please follow the system instructions to convert the Txt file to a CSV.")

# Clean the CSV strings
cleaned_syllabus_csv_string = clean_csv_string(syllabus_csv_string)
cleaned_weekly_schedule_csv_string = clean_csv_string(weekly_schedule_csv_string)

# Parse cleaned CSV strings to DataFrames
syllabus_df = csv_string_to_dataframe(cleaned_syllabus_csv_string)
weekly_schedule_df = csv_string_to_dataframe(cleaned_weekly_schedule_csv_string)

#Convert to proper CSV
syllabus_df.to_csv('Syllabus.csv', index=False)
weekly_schedule_df.to_csv('Weekly Schedule.csv', index=False)

# Load CSV files as pandas DataFrames
class_syllabus_df = pd.read_csv('Syllabus.csv')
weekly_schedule_df = pd.read_csv('Weekly Schedule.csv')
student_grades_df = pd.read_csv('Grades.csv')
professor_information_df = pd.read_csv('Professors.csv')

# Convert DataFrames to Documents
class_syllabus_doc = Document(page_content=class_syllabus_df.to_string(), name='Class Syllabus')
weekly_schedule_doc = Document(page_content=weekly_schedule_df.to_string(), name='Weekly Schedule')
student_grades_doc = Document(page_content=student_grades_df.to_string(), name='Student Grades')
professor_information_doc = Document(page_content=professor_information_df.to_string(), name='Professor Information')

# Prepare the list of documents
docs = [
    class_syllabus_doc,
    weekly_schedule_doc,
    student_grades_doc,
    professor_information_doc
]

In [2]:
#Sample queries tested:
print("\n")
result = run_query("What is the syllabus for the subject taught in 3rd period on Monday?")
print(result)
print("\n")
result = run_query("Which of the Mathematics professors are male?")
print(result)
print("\n")
result = run_query("What is the average age of professors who teach Mathematics in the weekly schedule?")
print(result)
print("\n")
result = run_query("What is the average grade for the subject taught in 1st period on Tuesday?")
print(result)
print("\n")
result = run_query("What days and periods is the subject with the lowest average grade taught?")
print(result)
print("\n")
result = run_query("What is the subject with the lowest average grade? Find its syllabus.")
print(result)



The subject taught in the 3rd period on Monday is Chemistry. 

The syllabus for 10th grade Chemistry is as follows:

In 10th grade Chemistry, students will explore the fundamental principles of the physical and chemical properties of matter. They will learn about the periodic table, chemical reactions, and atomic structure. Students will be introduced to laboratory techniques and safety procedures and perform experiments to reinforce theoretical concepts. The course will also focus on real-world applications of chemistry in fields such as medicine, environmental science, and engineering. By the end of the course, students will have a strong foundation in chemistry that will enable them to pursue further study in the field.


The Mathematics professors listed in the weekly schedule are David Kim, Ava Chen, Sophia Hernandez, and Jasmine Gomez. 

From the professor information:
- David Kim is male.
- Ava Chen is female.
- Sophia Hernandez is female.
- Jasmine Gomez is female.

Therefore

In [3]:
#Custom Inputs and Queries:
if __name__ == "__main__":
    while True:
        user_query = input("Please enter your query (if you want to exit at any time just type 'exit'): ")
        if user_query == 'exit':
            print("Exiting the query session.")
            break
        print("\n")
        result = run_query(user_query)
        print(result)

Exiting the query session.


This was a fantastic exercise and I had so much fun fleshing out my skills and creativity! -Maxwell Palance

The subject taught in the 3rd period on Monday is Chemistry. Here is the syllabus for 10th grade Chemistry:

In 10th grade Chemistry, students will explore the fundamental principles of the physical and chemical properties of matter. They will learn about the periodic table, chemical reactions, and atomic structure. Students will be introduced to laboratory techniques and safety procedures and perform experiments to reinforce theoretical concepts. The course will also focus on real-world applications of chemistry in fields such as medicine, environmental science, and engineering. By the end of the course, students will have a strong foundation in chemistry that will enable them to pursue further study in the field.


The Mathematics professors listed in the weekly schedule are David Kim, Ava Chen, Sophia Hernandez, and Jasmine Gomez. 

From the professor information:
- David Kim is male.
- Ava Chen is female.
- Sophia Hernandez is female.
- Jasmine Gomez is female.

Therefore, the male Mathematics professor is David Kim.


To find the average age of professors who teach Mathematics, we need to identify all the professors who teach Mathematics from the weekly schedule and then calculate the average of their ages.

From the weekly schedule, the professors who teach Mathematics are:
- David Kim (1st Period on Monday and 4th Period on Thursday)
- Ava Chen (4th Period on Tuesday)
- Sophia Hernandez (2nd Period on Wednesday)
- Jasmine Gomez (3rd Period on Friday)

Now, let's find their ages from the professor information:
- David Kim: 35 years old
- Ava Chen: 31 years old
- Sophia Hernandez: 33 years old
- Jasmine Gomez: 42 years old

To calculate the average age:
\[ \text{Average Age} = \frac{35 + 31 + 33 + 42}{4} \]

\[ \text{Average Age} = \frac{141}{4} \]

\[ \text{Average Age} = 35.25 \]

The average age of professors who teach Mathematics is 35.25 years.


The subject taught in 1st period on Tuesday is Physics.

To find the average grade for Physics, we will calculate the mean of the Physics grades for all students:

Physics grades:
- Student 1: 80
- Student 2: 88
- Student 3: 82
- Student 4: 90
- Student 5: 85
- Student 6: 82
- Student 7: 88
- Student 8: 90
- Student 9: 78
- Student 10: 88

Sum of Physics grades = 80 + 88 + 82 + 90 + 85 + 82 + 88 + 90 + 78 + 88 = 851

Number of students = 10

Average grade = Sum of Physics grades / Number of students = 851 / 10 = 85.1

The average grade for Physics is 85.1.


The subject with the lowest average grade is English Language Arts, with an average grade of 78.3.

Here are the days and periods when English Language Arts is taught:

- Monday: 4th Period
- Tuesday: 3rd Period
- Wednesday: 1st Period
- Thursday: 6th Period
- Friday: 7th Period


To determine the subject with the lowest average grade, we need to calculate the average grade for each subject and then identify the lowest one.

Here are the average grades for each subject:

- Chemistry: (85 + 90 + 78 + 92 + 87 + 80 + 95 + 82 + 88 + 90) / 10 = 86.7
- Mathematics: (92 + 85 + 89 + 93 + 85 + 90 + 87 + 82 + 92 + 85) / 10 = 88
- Physical Education: (78 + 80 + 82 + 88 + 80 + 82 + 85 + 75 + 85 + 80) / 10 = 81.5
- Foreign Language: (90 + 92 + 85 + 87 + 89 + 92 + 78 + 85 + 90 + 92) / 10 = 88
- Art: (95 + 93 + 90 + 85 + 90 + 95 + 93 + 90 + 95 + 93) / 10 = 91.9
- Economics: (87 + 75 + 92 + 85 + 88 + 90 + 87 + 85 + 92 + 75) / 10 = 85.6
- Physics: (80 + 88 + 82 + 90 + 85 + 82 + 88 + 90 + 78 + 88) / 10 = 85.3
- English Language Arts: (75 + 80 + 78 + 82 + 75 + 80 + 78 + 80 + 75 + 80) / 10 = 78.3

The subject with the lowest average grade is English Language Arts with an average grade of 78.3.

Here is the syllabus for 10th grade English Language Arts:

"In 10th grade English Language Arts, students will explore a variety of literary genres and develop critical reading, writing, and analytical skills. They will analyze and interpret texts from different time periods and cultural backgrounds, and explore themes such as identity, power, and relationships. The course will emphasize the development of strong writing skills, including argumentative, narrative, and expository writing. Students will also engage in discussions and presentations to enhance their communication skills. By the end of the course, students will have a deeper understanding of literature and improved proficiency in reading and writing."